In [ ]:
!pip install --upgrade langchain_openai
!pip install --upgrade openai
!pip install tiktoken
!pip install httpx==0.27.2
!pip install openai==1.55.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.55.3
    Uninstalling openai-1.55.3:
      Successfully uninstalled openai-1.55.3
  Using cached openai-1.55.3-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.55.3-py3-none-any.whl (389 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.61.1
    Uninstalling openai-1.61.1:
      Successfully uninstalled openai-1.61.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.5 requires openai<2.0.0,>=1.58.1, but you have openai 1.55.3 which is incompatible.


In [ ]:
import os
from pathlib import Path
import copy
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from langchain_openai import ChatOpenAI
import tiktoken

from collections import defaultdict
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from torch.utils.data import DataLoader, Dataset
from torch import FloatTensor, LongTensor

import sys
sys.path.append('/content/drive/MyDrive/ex2024/')

# colab 関連
from google.colab import files
from google.colab import userdata

llm_model_name = ["gpt-4o-mini-2024-07-18","plamo-1.0-prime"]
# os.environ['OPENAI_API_KEY'] = 'yyyyyyyyyyyy'
# os.environ['API_HOST']='https://platform.preferredai.jp'
import openai
from openai import OpenAI
openai.organization = "xxxxx"
openai.api_key = "xxxxxxxxxxxxxxxxxxxxxx"
client = OpenAI(api_key=openai.api_key, organization=openai.organization)
# llm = ChatOpenAI(
#     base_url=f"{os.environ['API_HOST']}/api/completion/v1",
#     model=llm_model_name[0],
#     streaming=False,
#     # other params...,
#     temperature=0.0
# )



In [ ]:
def load_jsonl(path: Path | str) -> pd.DataFrame:
    path = Path(path)
    return pd.read_json(path, lines=True)


def load_json(path: Path | str) -> dict:
    path = Path(path)
    with path.open() as f:
        data = json.load(f)
    return data

In [ ]:
# -*- coding: utf-8 -*-
class Args():
    #BERT訓練済みモデル
    bert_pretrained_model_name = "cl-tohoku/bert-base-japanese-v3"
    max_seq_len = 512 #BERT入力列最大値
    batch_size = 32

    seed: int = 42


    sampling_flag = {'train': True, 'val': False, 'test': True}
    sampling_rate = {'train': 0.35, 'val': 1.0, 'test': 0.1}

    #livedoorニュースコーパスデータセットへのパス
    dataset_dir: Path = Path('/content/drive/MyDrive/ex2024/livedoor/datasets/livedoor/')

    #要約辞書保存先
    summary_dict = defaultdict(lambda: defaultdict(list))
    output_model_dir: Path = Path('/content/drive/MyDrive/ex2024/livedoor/datasets/livedoor/summary/')

In [ ]:
schema={
  "$schema": "https://json-schema.org/draft/2020-12/schema",
  "type": "object",
  "properties": {
    "results": {
      "type": "object",
      "properties": {
        "1st": {
          "type": "integer",
          "description": "最もふさわしいラベル番号",
          "minimum": 0,
          "maximum": 8
        },
        "2nd": {
          "type": "integer",
          "description": "2 番目にふさわしいラベル番号",
          "minimum": 0,
          "maximum": 8
        },
        "3rd": {
          "type": "integer",
          "description": "3 番目にふさわしいラベル番号",
          "minimum": 0,
          "maximum": 8
        }
      },
      "required": ["1st", "2nd", "3rd"],
      "additionalProperties": False,
      "allOf": [
        {
          "not": {
            "anyOf": [
              { "properties": { "1st": { "const": { "$data": "2nd" } } } },
              { "properties": { "1st": { "const": { "$data": "3rd" } } } },
              { "properties": { "2nd": { "const": { "$data": "3rd" } } } }
            ]
          }
        }
      ]
    }
  },
  "required": ["results"],
  "additionalProperties": False
}


few_shot_examples = [
    {"role": "user", "content": 'タイトル: "30代女性の新トレンド、週末ソロ旅行が人気"\n本文: "一人の時間を大切にする女性が増え、週末に気軽に行けるソロ旅行がトレンドになっている。SNSでも多くの体験談がシェアされている。"'},
    {"role": "assistant", "content": '{"results": {"1st": 0, "2nd": 5, "3rd": 8}}'},

    {"role": "user", "content": 'タイトル: "パスワード管理の新常識、生成型AIでセキュリティ強化"\n本文: "最近の研究によると、AIを活用したパスワード生成が従来の手動作成よりも安全性が高いことが分かった。専門家は定期的な更新を推奨している。"'},
    {"role": "assistant", "content": '{"results": {"1st": 1, "2nd": 8, "3rd": 6}}'},

    {"role": "user", "content": 'タイトル: "最新コードレス掃除機、吸引力テストで驚きの結果"\n本文: "人気メーカーの新型コードレス掃除機が吸引力テストで高評価を獲得。軽量ながらパワフルな性能が話題になっている。"'},
    {"role": "assistant", "content": '{"results": {"1st": 2, "2nd": 6, "3rd": 1}}'},

    {"role": "user", "content": 'タイトル: "大人の男性に人気の腕時計ブランドランキング発表"\n本文: "最新の調査で、30代以上の男性に最も人気のある腕時計ブランドランキングが発表された。高級時計からカジュアルなものまで幅広い選択肢がある。"'},
    {"role": "assistant", "content": '{"results": {"1st": 3, "2nd": 5, "3rd": 8}}'},

    {"role": "user", "content": 'タイトル: "話題の新作SF映画、公開3日で興行収入10億円突破"\n本文: "未来都市を舞台にした話題のSF映画が公開初日から大ヒット。迫力の映像美とストーリー展開が観客を魅了している。"'},
    {"role": "assistant", "content": '{"results": {"1st": 4, "2nd": 8, "3rd": 3}}'},

    {"role": "user", "content": 'タイトル: "今年の春メイクトレンドは『ナチュラルグロウ』"\n本文: "美容専門家によると、今年の春のメイクトレンドはナチュラルなツヤ感を生かした『ナチュラルグロウ』。SNSでも多数のメイク動画が投稿されている。"'},
    {"role": "assistant", "content": '{"results": {"1st": 5, "2nd": 8, "3rd": 0}}'},

    {"role": "user", "content": 'タイトル: "次世代スマホ、AIカメラ機能が大幅進化"\n本文: "最新スマートフォンのAIカメラ機能が強化され、夜間撮影の精度が向上。ユーザーからも『プロ並みの写真が撮れる』と好評だ。"'},
    {"role": "assistant", "content": '{"results": {"1st": 6, "2nd": 1, "3rd": 2}}'},

    {"role": "user", "content": 'タイトル: "プロ野球の新星、デビュー戦で決勝ホームラン"\n本文: "今シーズン期待のルーキー選手がデビュー戦で劇的な決勝ホームランを放ち、チームの勝利に貢献した。"'},
    {"role": "assistant", "content": '{"results": {"1st": 7, "2nd": 8, "3rd": 3}}'},

    {"role": "user", "content": 'タイトル: "国内で新たな再生可能エネルギープロジェクトが始動"\n本文: "政府は最新の再生可能エネルギープロジェクトを発表し、2030年までの温室効果ガス削減目標の達成を目指す。"'},
    {"role": "assistant", "content": '{"results": {"1st": 8, "2nd": 1, "3rd": 4}}'}
]

In [ ]:
label_list = """
0: 独女通信 → 独身女性向けのライフスタイル・恋愛・ファッション
1: IT ライフハック → IT・テクノロジー・ハウツー
2: 家電チャンネル → 家電・ガジェット・レビュー
3: livedoor HOMME → 男性向けライフスタイル・ファッション・趣味
4: MOVIE ENTER → 映画・エンタメ・レビュー
5: Peachy → 女性向けライフスタイル・美容・恋愛
6: エスマックス → スマホ・モバイル関連情報
7: Sports Watch → スポーツニュース・試合結果・選手動向
8: トピックニュース → 国内外の社会・時事ニュース
"""

In [ ]:
def make_prompt(title, body):
    """ 記事のタイトルと本文を与えると、いい感じにプロンプトを返す """
    prompt = f"""
    与えられた記事の「タイトル」と「本文」に基づいて、ふさわしい **上位3つのラベル** を 9つのラベルから厳密に選定してください。
    ラベルは **1st, 2nd, 3rd** の順位で重要度順に並べ、以下に示す JSON Schema に厳密に従って **JSON 形式** で出力してください。
    ### **ラベル一覧**
    {label_list}

    ### **JSON Schema**
    {schema}

    ### **出力ルール**
    - **厳密に 3 つのラベルを選ぶこと**
    - **順位は関連度の高い順**
    - **出力形式を厳格に守ること 特に results という key を必ず持ち、子要素として 1st・2nd・3rd を持たせること。**
    - **JSON 形式以外の文字は一切応答に含めないこと**

    ### **記事情報**
    - **タイトル**: {title}
    - **本文**: {body}

    **JSON 出力**
    """

    return prompt


In [ ]:


# 使用するモデルに対応するトークナイザを取得
encoding = tiktoken.get_encoding("cl100k_base")

def get_summary(batch, batch_size=32):
    res = defaultdict(lambda: defaultdict(list))


    # 最大トークン数を設定（例: 4096 - 出力用トークンを残して 2048 トークン）
    input_max_tokens = 3600*3
    for i in range(batch_size):
        input_tokens = encoding.encode(batch['body'][i])
        num_tokens = len(input_tokens)
        body = batch['body'][i]
        if num_tokens > input_max_tokens:
            # トークンを制限内に収めるためにカット
            input_tokens = input_tokens[:input_max_tokens]
            # トークンを再びテキストに変換
            trimmed_prompt = encoding.decode(input_tokens)
            body = trimmed_prompt
        messages=[
            {"role": "system", "content": "あなたは優秀なAIアシスタントです。"},
            *few_shot_examples,
            {"role": "user", "content": make_prompt(batch['title'][i], body)}
        ]
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            temperature = 0.0
            )

        res[batch['category-ids'][i]] = response.choices[0].message.content
        print(batch['title'][i][:100])
        print("正解: {0}".format(batch['category-ids'][i]))
        print("---------------")
        print(res[batch['category-ids'][i]][:100])
        print("===============")

    return res


In [ ]:
class Experiment():
    def __init__(self, _args: Args):
        #実験設定
        self.args = _args
        self.phase = ['test']
        #データローダー
        self.dataloader = defaultdict(lambda: defaultdict(list))
        #データセット(DataFrame)
        self.dataset = defaultdict(lambda: defaultdict(list))
        for p in self.phase:
            self.dataloader[p] = self.load_dataset(phase = p)


    def load_dataset(self, phase):
        path = self.args.dataset_dir / f'{phase}.jsonl'
        original = load_jsonl(path)

        data = original.to_dict(orient="records")
        self.dataset[phase] = pd.DataFrame(data.copy())
        return DataLoader(data, collate_fn=self.collate_fn, batch_size=self.args.batch_size,shuffle=False,num_workers=2,pin_memory=True)

    # カスタムcollate_fnの定義
    def collate_fn(self, batch):
        title = [data['title'] for data in batch]
        body = [data['body'] for data in batch]
        #category-idから元のデータを参照できるようにする
        category_ids = [data['category-id'] for data in batch]
        return {'title': title, 'body': body, 'category-ids': category_ids}

    def category_id2article(self, phase, category_id):
        return self.dataset[phase][self.dataset[phase]['category-id'] == category_id]


    def train(self):
        for phase in ['test']:
            print(phase + " start!!\n")
            dataloader_size = len(self.dataloader[phase])
            cnt = 0
            for batch in self.dataloader[phase]:
                print("{}/{}\n".format(cnt,dataloader_size))
                # print(batch['title'], batch['category-id'])
                cnt = cnt + 1
                self.args.summary_dict.update(get_summary(batch, self.args.batch_size))

        # JSONファイルに保存
        with open('/content/drive/MyDrive/ex2024/livedoor/datasets/livedoor/summary/get_label_rank_test_4omini_few', 'w') as json_file:
            json.dump(self.args.summary_dict, json_file)

    def save_model(self):
        torch.save(self.net.state_dict(), self.new_model_path)
        print('\nbest score updated, Pytorch model was saved!! val_acc:{}\n'.format(self.history.best['val']['acc']))


    def run(self):
        print("ex start")
        self.label_distribution()
        self.train()

    def label_distribution(self):
        # ラベルごとのデータ数を集計
        for p in self.phase:
            label_counts = self.dataset[p]['label'].value_counts()
            print(p, label_counts)

In [ ]:
ex = Experiment(_args = Args())
ex.run()
print('---finish---')

ex start
test label
0    98
5    97
1    95
2    86
6    80
7    79
4    77
8    76
3    48
Name: count, dtype: int64
test start!!

0/23

実現可能? 夢物語? 月にメガソーラー構想で世界の総電力が発電可能に【話題】
正解: kaden-channel-6162787
---------------
{"results": {"1st": 8, "2nd": 1, "3rd": 0}}
周辺地域の情報をゲット「ご当地ニュースbyロケタッチ新聞」リリース
正解: it-life-hack-6541754
---------------
{
  "results": {
    "1st": 8,
    "2nd": 0,
    "3rd": 1
  }
}
「信じられない!」 槙野の浦和移籍に非難轟々
正解: sports-watch-6184005
---------------
{"results": {"1st": 7, "2nd": 8, "3rd": 3}}
【Sports Watch】低迷する巨人の次期監督候補に江川氏!?
正解: sports-watch-5720704
---------------
{"results": {"1st": 7, "2nd": 8, "3rd": 3}}
ドクター圭子の知らなきゃケガするオトナの医学 vol.3〜Presented by ゆるっとcafe〜
正解: dokujo-tsushin-5455376
---------------
{"results": {"1st": 5, "2nd": 0, "3rd": 8}}
『HEROES/ヒーローズ』最大の主人公が発狂「心を打つ、悲しげな役を演じたくなかった」
正解: movie-enter-6618222
---------------
{"results": {"1st": 4, "2nd": 0, "3rd": 8}}
AKB48に言及したノエル・ギャラガー、意図せず誤解を招く?
正解: topic-news-6602491
---------------
{"results": {"1st": 4, "2nd": 8, 